# Datenvorbereitung
Dieses Notebook soll dazu dienen die Daten aus dem TIGER Korpus auszulesen und für die weiteren Schritte vorzubereiten. Diese Datenvorbereritung basiert auf https://github.com/theodm/gender-assistenz/blob/master/results/tiger_extract.py wird allerdings an gewissen Punkten verbessert. Die Artikel werden bereits automatisch getrennt und ihrer Artikel-ID zugeordnet und die Autoren/Autorinnen der Artikel werden gefiltert, dies muss also nicht mehr manuell passieren.
## Datenauslesen
Zunächst müssen die Daten aus der XML ausgelesen werden, dabei sollen die als Männlich erkannten Wörter mit zwei Bindestrichen vor und nach dem Wort markiert werden. Die Daten sollen Artikelweise in eine Output Datei gelegt werden um dort manuell weiter bearbeitet zu werden. 

In [3]:
from lxml import etree as ET
import csv
import re
import pandas as pd
import spacy
import textdescriptives as td


Zunächst wird die Datei, welche die Sätze ihren Artikeln zurodnet ausgelesen und in einem dictionary gespeichert

In [4]:
sentences_to_documents_dict = {}
sentences_to_type_dict = {}
with open("sentences.tsv", "r", encoding="utf-8") as file:
    reader = csv.reader(file, delimiter="\t")  
    for row in reader:
        if len(row) >= 3:  # Sicherstellen, dass die Zeile mindestens drei Spalten hat
            sentence_id = row[1]  # ID als Schlüssel (Spalte B)
            article_id = row[0]  # Erste Spalte als Wert (Spalte A)
            _type = row[2]
            sentences_to_documents_dict[sentence_id] = article_id
            sentences_to_type_dict[sentence_id] = _type
            

Daraufhin wird die Output Datei eingelesen und in Artikel sortiert abgespeichert. Hierbei werden die potentiell generischen Maskulina mit zwei "--" vor und hinter dem Wort markiert.

In [5]:
# Das ist die ID des ersten Artikels diese kann hier ausgelesen werden
previous_article = sentences_to_documents_dict["1"]
counter = 0
with open("preparedData.txt", "w", encoding="utf-8") as outputFile:
    for event, s_tag in ET.iterparse("tiger_release_aug07.corrected.16012013.xml", events=("end",), tag=f"s"):
        tag_id=s_tag.get("id")[1:]
        current_article = sentences_to_documents_dict[tag_id]
        if current_article != previous_article:
            outputFile.write(f"\n---\n\nArtikelId: {current_article}\n") 
            previous_article = current_article
        # Hinter diesem type versteckt sich der 
        if sentences_to_type_dict[tag_id] == "Meta":
            continue
        graph_tag = s_tag.find(f"graph")
        terminals_tag = graph_tag.find(f"terminals")
        t_tags = terminals_tag.findall(f"t")

        sentence = []
        for t_tag in t_tags:
            word = {}
    
            word["word"] = t_tag.attrib["word"]
            word["pos"] = t_tag.attrib["pos"]
            word["number"] = t_tag.attrib["number"]
            word["gender"] = t_tag.attrib["gender"]
            word["case"] = t_tag.attrib["case"]
    
            if (word["gender"] in ["Masc", "*"] or (word["gender"] in ["Neut"] and word["number"] == "Plur")) and word["pos"] in ["PDS", "PIS", "PPER", "PPOSS", "PRELS", "PWS", "NN"]:
                word["word"] = "--" + word["word"] + "--"
    
            sentence.append(word)
    
        outputFile.write(" ".join(x["word"] for x in sentence).replace(" ,", ",").replace(" ;", ";").replace(" .", ".").replace(" :", ":").replace(" ?", "?").replace(" .", ".").replace(" !", "!").replace("`` ", "\"").replace(" `` ", "\"").replace(" ``", "\"").replace("'' ", "\"").replace(" '' ", "\"").replace(" ''", "\"").replace("( ", "(").replace(" )", ")") + "\n")


FileNotFoundError: [Errno 2] No such file or directory: 'tiger_release_aug07.corrected.16012013.xml'

In der soeben erstellten Datei muss nun manuell für die ob es sich um ein generisches Maskulinum handelt und somit korrekturbedürftig ist oder nicht. Ein Ausdruck gilt als generisches Maskulinum, wenn eine maskuline Wortform verwendet wird, um eine gemischtgeschlechtliche oder geschlechtsunspezifische Gruppe zu bezeichnen, ohne eine explizite geschlechtliche Markierung zu enthalten. Typischerweise betrifft dies maskuline Substantive wie Student, Arzt oder Bürger, die in einem allgemeinen Sinne für alle Geschlechter stehen sollen. Entscheidend ist der kontextuelle Gebrauch, bei dem keine eindeutige geschlechtsspezifische Zuordnung erkennbar ist und das Maskulinum als Standard-Form fungiert. Die Korrekturbedürftigen Formen werden hierbei mit einem "!" die nicht korrektur bedürftigen mit einem "\" geprefixed. Für die ersten x Artikel werden die bereits in https://github.com/theodm/gender-assistenz/blob/master/results/demofile3.txt bestehenden manuellen Annotationen übernommen.
## Weiterverarbeitung der manuellen Daten
In diesem Abschnitt werden die manuell vorbereiteten Daten in ein Dataframe eingelesen dabei werden sie um den unmarkierten Text ergänzt und die einzelnen Sätze der Texte werden in eine eigene Liste gespeichert.

In [6]:
with open('preparedData_manual.txt', 'r', encoding='utf-8') as f:
    content = f.read()

# Aufteilen in Blöcke anhand des Trennzeichen
artikel_blocks = content.strip().split('\n\n---\n\n')

data = []

for block in artikel_blocks:
    lines = block.strip().split('\n')
    artikel_id = lines[0]
    text = '\n'.join(lines[1:])  # Falls der Artikeltext mehrzeilig ist
    data.append((artikel_id, text))

df = pd.DataFrame(data, columns=['ArtikelId', 'Text'])


Weitere Datenfelder zu dem Text ergänzen. Den ganzen Text unmarkiert und sowohl Einzelne Sätze als Liste sowohl markiert als auch unmarkiert. Diese Funktion soll einen Fehler werfen wenn unmarkierte vorkommen gefunden werden um manuell in der Datei korrigieren zu können.

In [7]:
def clean_and_check_article(article):
    #
    # Parst einen Artikel im
    #
    t = article
    ci = 0
    clean_article = ""


    number_of_ignored_chars = 0
    try:
        while ci < len(article)-2:
            if article[ci] in ("!", "\\") and article[ci+1:ci+3] == "--":
                # Zum Anfang des Wortes springen
                ci = ci + 3
                while True:
                    if article[ci:ci+2] == "--":
                        ci = ci + 2
                        break
                    clean_article = clean_article + article[ci]
                    ci = ci + 1
                continue
    
            if t[ci] == "-" and t[ci + 1] == "-":
                raise Exception("-- found: " + article[ci-10:ci+20])
    
            clean_article = clean_article + article[ci]
            ci = ci + 1
    
        return clean_article
    except: 
        print(article)
        raise Exception("-- found: " + article[ci-10:ci+20])



In [8]:
df['Text_unmarked'] = df['Text'].apply(clean_and_check_article)
df['Sentences_marked'] = df['Text'].apply(lambda x: x.split('\n'))
df['Sentences_unmarked'] = df['Text_unmarked'].apply(lambda x: x.split('\n'))

Um später detaillierte Analysen zu ermöglichen werden an dieser Stelle noch Metriken ergänzt um den schreibstil und die komplexität der Artikel besser zu beschreiben. 

In [10]:
nlp = spacy.load("de_core_news_lg")
nlp.add_pipe("textdescriptives/dependency_distance")  # fügt alle Features hinzu

def extract_features(text):
    doc = nlp(text)
    return {
        "dependency_distance": doc._.dependency_distance,
    }

features = df["Text_unmarked"].apply(extract_features)

df["dependency_distance"] = features.apply(lambda x: x["dependency_distance"])

# Optional: df.head() zur Kontrolle
df.head(5)

/opt/anaconda3/envs/spacy310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/spacy310/lib/python3.10/site-packages/IPython/core/formatters.py:406: FutureWarning: Index.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()
/opt/anaconda3/envs/spacy310/lib/python3.10/site-packages/IPython/core/formatters.py:406: FutureWarning: RangeIndex.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()


,ArtikelId,Text,Text_unmarked,Sentences_marked,Sentences_unmarked,dependency_distance
0,ArtikelId: 0001_0001,"""Ross Perot wäre vielleicht ein prächtiger \--...","""Ross Perot wäre vielleicht ein prächtiger Dik...","[""Ross Perot wäre vielleicht ein prächtiger \-...","[""Ross Perot wäre vielleicht ein prächtiger Di...",{'dependency_distance_mean': 2.965713889246949...
1,ArtikelId: 0001_0002,IBM und Siemens gelten nicht mehr als Schimpfw...,IBM und Siemens gelten nicht mehr als Schimpfw...,[IBM und Siemens gelten nicht mehr als Schimpf...,[IBM und Siemens gelten nicht mehr als Schimpf...,{'dependency_distance_mean': 2.782908048111762...
2,ArtikelId: 0001_0003,Wechselspiel von Dramatisierung und Ignoranz\n...,Wechselspiel von Dramatisierung und Ignoranz\n...,"[Wechselspiel von Dramatisierung und Ignoranz,...","[Wechselspiel von Dramatisierung und Ignoranz,...",{'dependency_distance_mean': 3.174690061989753...
3,ArtikelId: 0001_0004,Im \--Blickpunkt--:\nErmittlungen gegen \--Aut...,Im Blickpunkt:\nErmittlungen gegen Autonome\nS...,"[Im \--Blickpunkt--:, Ermittlungen gegen \--Au...","[Im Blickpunkt:, Ermittlungen gegen Autonome, ...",{'dependency_distance_mean': 3.137275306681283...
4,ArtikelId: 0001_0005,Für ehrliche !--Kunden-- ist es ein \--Schock-...,Für ehrliche Kunden ist es ein Schock\nZahl de...,[Für ehrliche !--Kunden-- ist es ein \--Schock...,"[Für ehrliche Kunden ist es ein Schock, Zahl d...",{'dependency_distance_mean': 2.704419452574911...



An dieser Stelle sind die Daten soweit vorbereitet um im [DataExploration](./DataExploration.ipynb) weiter untersucht zu werden.

In [11]:
df.to_pickle('data_prepared_frame.pkl')

,ArtikelId,Text,Text_unmarked,Sentences_marked,Sentences_unmarked
0,ArtikelId: 0001_0001,"""Ross Perot wäre vielleicht ein prächtiger \--...","""Ross Perot wäre vielleicht ein prächtiger Dik...","[""Ross Perot wäre vielleicht ein prächtiger \-...","[""Ross Perot wäre vielleicht ein prächtiger Di..."
1,ArtikelId: 0001_0002,IBM und Siemens gelten nicht mehr als Schimpfw...,IBM und Siemens gelten nicht mehr als Schimpfw...,[IBM und Siemens gelten nicht mehr als Schimpf...,[IBM und Siemens gelten nicht mehr als Schimpf...
2,ArtikelId: 0001_0003,Wechselspiel von Dramatisierung und Ignoranz\n...,Wechselspiel von Dramatisierung und Ignoranz\n...,"[Wechselspiel von Dramatisierung und Ignoranz,...","[Wechselspiel von Dramatisierung und Ignoranz,..."
3,ArtikelId: 0001_0004,Im \--Blickpunkt--:\nErmittlungen gegen \--Aut...,Im Blickpunkt:\nErmittlungen gegen Autonome\nS...,"[Im \--Blickpunkt--:, Ermittlungen gegen \--Au...","[Im Blickpunkt:, Ermittlungen gegen Autonome, ..."
4,ArtikelId: 0001_0005,Für ehrliche !--Kunden-- ist es ein \--Schock-...,Für ehrliche Kunden ist es ein Schock\nZahl de...,[Für ehrliche !--Kunden-- ist es ein \--Schock...,"[Für ehrliche Kunden ist es ein Schock, Zahl d..."
